In [13]:
from collections import defaultdict
import pandas as pd

from surprise.model_selection import KFold, cross_validate

from surprise import KNNBasic

from surprise import SVD
from surprise import SVDpp
from surprise import  NMF
from surprise import NormalPredictor, BaselineOnly
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV


from model.data_interface.data import *

In [14]:
data = load_data(data_path="../data/dicoding_user_item_rating.gzip")
print(data.head())

data_input = data.loc[:, ["user_id", "course_id", "rating"]]
surprise_input = reader_data(data=data_input,
                             cols=["user_id", "course_id", "rating"],
                             scale=True,
                             model="surprise")

Index(['user_id', 'course_id', 'graduated_at', 'rating'], dtype='object')
   user_id  course_id        graduated_at  rating
0   623699         14 2020-03-10 11:45:50       5
1   406371         14 2020-03-10 08:44:09       4
2     1946         14 2020-03-11 13:24:10       5
3   186713         14 2020-03-11 17:36:04       4
4   462580         14 2020-03-11 08:43:36       4


In [15]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [16]:
def precision_recall_at_k(predictions, k=10, threshold=4.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [17]:
trainset = surprise_input.build_full_trainset()

testset  = trainset.build_anti_testset()

In [18]:
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.

predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

In [19]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    if uid == int("2447675"):
        print(uid, [iid for (iid, _) in user_ratings])

2447675 [615, 159, 428, 169, 423, 292, 403, 219, 387, 86]


In [20]:
kf = KFold(n_splits=5)
algo = SVD()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.8746445421703163
0.8917465981898972
0.874188865092925
0.8914086733425233
0.8731406583529947
0.8911931889831644
0.8767020484618528
0.8937477910635374
0.8739442849445503
0.8913595288453549


In [21]:
kf = KFold(n_splits=5)
algo = SVDpp()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.8720813098042993
0.8879795211333411
0.8721297409991186
0.8887686098414989
0.8718061674008815
0.8888691054123999
0.8740667434384161
0.8897912183035257
0.8693280202282082
0.8860407337020473


In [22]:
 
kf = KFold(n_splits=5)
algo = NormalPredictor()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.7627772841166545
0.7379483024347224
0.7632377105024807
0.7385964631041223
0.7624376430895287
0.7374217681610152
0.7640318892034103
0.7385643727976893
0.7637126018123198
0.7376406767498748


In [23]:
 
kf = KFold(n_splits=5)
algo = BaselineOnly()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

Estimating biases using als...
0.8756038414594985
0.8938080240432572
Estimating biases using als...
0.8777210638673543
0.8962468595245356
Estimating biases using als...
0.8768973214285715
0.8956550838301273
Estimating biases using als...
0.8757322929171673
0.8941835158305745
Estimating biases using als...
0.8754158853791046
0.894218767767376


In [24]:

kf = KFold(n_splits=5)
algo = NMF()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.5419032736306446
0.4987749140976691
0.6070854585699161
0.5698953633007136
0.5942136178374491
0.557370538968477
0.5849086576648131
0.5457599485796464
0.5578595773687798
0.5150176495918106
